<div align="Center">

# Orthogonalization and Least Squares Methods
## Matrix Computations (AS1209)
### Utkarsh Tailor (2022BTech106)
### Swastik Kulshreshtha (2022BTech105)
### Saurabh Saini (2022BTech093)
### Rajat Paliwal (2022BTech081)
### Rahul Yadav (2022BTech079)
#### Institute of Engineering and Technology, JK Lakshmipat University

</div>
<hr>

##### Importing Libraries

In [ ]:
from typing import Callable, Tuple

import numpy as np
import matplotlib.pyplot as plt

##### Utilities

In [ ]:
def getRectIdentity(n: int, m: int) -> np.ndarray:
    assert n > 0 and m > 0, "n and m must be greater than 0"
    mat = np.zeros((n, m))
    for i in range(min(n, m)):
        mat[i][i] = 1
    return mat

def getQR(mat: np.ndarray, method: Callable[[np.ndarray], Tuple[np.ndarray, np.ndarray]]) -> tuple[np.ndarray, np.ndarray]:
    q, r = method(mat)
    print("Q:")
    print(q)
    print("\nR:")
    print(r)
    print("\nQR:")
    print(q @ r)
    return q, r

In [ ]:
def plot2Dpoints(matrix, ls='--', lw=1.2, colors=None):
    x_points, y_points = matrix
    size = len(x_points)
    colors = ['red', 'blue', 'orange', 'green'] if None else colors
    for i in range(size):
        plt.plot(x_points[i], y_points[i], color=colors[i], marker='o')
        plt.plot(
            [x_points[i], x_points[(i+1) % size]], 
            [y_points[i], y_points[(i+1) % size]], 
            color=colors[i], 
            linestyle=ls, linewidth=lw
        )

def plot3Dpoints(matrix, ls='--', lw=1.2, colors=None):
    x_points, y_points, z_points = matrix
    size = len(x_points)
    colors = ['red', 'blue', 'orange', 'green'] if None else colors
    for i in range(size):
        plt.plot(x_points[i], y_points[i], z_points[i], color=colors[i], marker='o')
        plt.plot(
            [x_points[i], x_points[(i+1) % size]], 
            [y_points[i], y_points[(i+1) % size]], 
            [z_points[i], z_points[(i+1) % size]],
            color=colors[i], 
            linestyle=ls, linewidth=lw
        )

##### Test Cases

In [ ]:
A = np.array([[2, -1, -2], [-4, 6, 3], [-4, -2, 8]])
B = np.array([[1, -4], [2, 3], [2, 2]])
C = np.array([[1, 1, 0], [0, 0, 3], [1, -1, 4]])

## Orthogonalization

<-- Definitions here -->

### QR Factorization

Given an $m\times n$ Matrix $A$, there exists an $m\times m$ Orthogonal Matrix $Q$ and an $m\times n$ Upper Triangular Matrix $R$, such that $A=QR$.

$$A_{m\times n}=Q_{m\times m}R_{m\times n}$$

<hr>

### Householder's Method

<hr>

Given a Non-Zero Vector $x\neq e_1$, the Householder Matrix $H$ defined by the Vector $V$,

$$\begin{align*}H&=I-\dfrac{2VV^T}{V^TV}\\ V&=x\pm ||x||_2e_1\text{ such that}\\ Hx&=\mp ||x||_2e_1\end{align*}$$

Here, $e_1$ is the First Vector of an Identity Matrix of Order $n\times n$.

<hr>

In [ ]:
def getHouseholderMatrix(mat: np.ndarray):
    x = mat[:, 0].reshape(-1, 1)
    v = np.copy(x)
    v[0, 0] += np.sign(x[0, 0]) * np.linalg.norm(x)
    v = v / np.linalg.norm(v)
    h = np.identity(mat.shape[0]) - 2 * (v @ v.T)
    return h

def getQRfromHouseholder(mat: np.ndarray):
    hhs = []
    cols = mat.shape[0]
    for i in range(min(mat.shape)):
        hcap = getHouseholderMatrix(mat[i:, i:])
        hi = np.identity(cols)
        hi[i:, i:] = hcap
        mat = hi @ mat
        hhs.append(hi)

    q = np.identity(cols)
    for h in hhs[::-1]:
        q = h @ q

    return q, mat

In [ ]:
qA, rA = getQR(A, getQRfromHouseholder)

In [ ]:
qB, rB = getQR(B, getQRfromHouseholder)

In [ ]:
qC, rC = getQR(C, getQRfromHouseholder)

#### Figures

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plot3Dpoints(A, colors=['red']*4)
plot3Dpoints(qA, colors=['blue']*4)
plot3Dpoints(rA, colors=['green']*4)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plot3Dpoints(C, colors=['red']*4)
plot3Dpoints(qC, colors=['blue']*4)
plot3Dpoints(rC, colors=['green']*4)
plt.show()

### Given's Method

<hr>

In [ ]:
def getGivensMatrix(n: int, m: int, i: int, k: int, theta: float = None, x: np.ndarray = None) -> np.ndarray:
    assert i != k, "i and k must be different"
    assert n > 0 and m > 0, "n and m must be greater than 0"
    assert i < n and i < m and k < n and k < m, "i and k must be less than n and m"
    assert not (theta is None and x is None), "Either theta or x must be provided"

    if theta is None:
        xi, xk = x[i], x[k]
        deno = np.sqrt(xi**2 + xk**2)
        cos = xi / deno
        sin = -xk / deno
    else:
        cos = np.cos(theta)
        sin = np.sin(theta)

    mat = getRectIdentity(n, m)
    mat[i][i] = cos
    mat[k][k] = cos
    mat[i][k] = sin
    mat[k][i] = -sin

    return mat

def getQRfromGivens(a: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    n, m = a.shape    
    givens = []
    lowerIndices = ((i, j) for i in range(1, n) for j in range(i))
    for i, j in lowerIndices:
        if a[i][j] == 0:
            continue
        g = getGivensMatrix(n, m, j, i, x=a.T[j])
        a = g.T @ a
        givens.append(g)

    q = getRectIdentity(n, m)
    for g in givens:
        q = q @ g

    return q, a

In [ ]:
qA, rA = getQR(A, getQRfromGivens)

In [ ]:
qC, rC = getQR(C, getQRfromGivens)

### Classical & Modified Gram Schmidt Method

<hr>

In [ ]:
import numpy as np

def getQRfromGramSchmidt(mat: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    m, n = mat.shape
    q = np.zeros((m, m))
    r = np.zeros((n, n))

    for j in range(n):
        v = mat[:, j]
        for i in range(j):
            r[i, j] = q[:, i].T @ mat[:, j]
            v = v.squeeze() - (r[i, j] * q[:, i])
        r[j, j] = np.linalg.norm(v)
        q[:, j] = (v / r[j, j]).squeeze()
    
    return q, r

In [ ]:
qA, rA = getQR(A, getQRfromGramSchmidt)

In [ ]:
qC, rC = getQR(C, getQRfromGramSchmidt)

### Least Squares Method

<hr>

In [ ]:
def leastSquareSolution(mat: np.ndarray, b: np.ndarray) -> np.ndarray:
    m, n = mat.shape
    if m >= n:
        aTrasnposeA = (mat.T @ mat)
        inv = np.linalg.inv(aTrasnposeA)
        invAtranspose = (inv @ np.transpose(mat))
        x = (invAtranspose @ b)
    else:
        ATransposea = (mat @ mat.T)
        inv = np.linalg.inv(ATransposea)
        invA = (np.transpose(mat) @ inv)
        x = (invA @ b)

    return x

In [ ]:
xA = leastSquareSolution(A, np.array([1, 2, 3]))
xB = leastSquareSolution(B, np.array([1, 2, 3]))

In [ ]:
xA, xB

<hr>